<a href="https://colab.research.google.com/github/codin-champ/ML-Projects/blob/main/Forest_Fire_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Libraries

In [ ]:
import pandas as pd
import numpy as np

# Reading Dataset

In [ ]:
filename = 'forestfires.csv' # upload this file everytime this is run
names = ['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area']

In [ ]:
df = pd.read_csv(filename, names=names)
print(pd.isnull(df).sum()) # returns the number of empty cells for each column

In [ ]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

## Discreptive Statistics

In [ ]:
filename = 'forestfires.csv' # upload this file everytime this is run
names = ['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area']
df = pd.read_csv(filename, names=names)

In [ ]:
print(df.shape)

In [ ]:
print(df.dtypes)

In [ ]:
print(df.head())

In [ ]:
print(df.describe()) # does not include the date and month column cause they are strings

In [ ]:
print(df.corr(method='pearson')) # correlation of each column with all others-->diagonal is always 1

In [ ]:
df.month.replace(('jan','feb', 'mar','apr','may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace =True)
print(df.head())

In [ ]:
df.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)
print(df.head())

## Visualizing the Data

In [ ]:
from matplotlib import pyplot as plt

### Histogram

In [ ]:
df.hist(sharex=False, sharey=False, xlabelsize=15, ylabelsize=15, color='orange', figsize=(15,15))
plt.suptitle("Histograms", y=1.00, fontweight='bold', fontsize=40)
plt.show()

### Probability Density Function(PDF)

In [ ]:
df.plot(kind='density',subplots=True, layout=(7,2), sharex=False, sharey=False, figsize=(15,15))
plt.suptitle("PDF", y=1.00, fontweight='bold', fontsize=40)
plt.show()

### Box and Whisker Plot

In [ ]:
df.plot(kind='box', subplots=False,layout=(4,4), sharex=False, sharey=False, fontsize=16,figsize=(10,10))
plt.suptitle("Box and Whisker",y=1.00, fontweight='bold', fontsize=40)
plt.show()

### Scatter Matrix
Yet to be completed

In [ ]:
from pandas.plotting._matplotlib import scatter_matrix
Axes = scatter_matrix(df,figsize(15,15))
plt.suptitle("Scatter Matrix", y=1.00,fontweight='bold', fontsize=30)
plt.rcParams['axes.labelsize']=15

# Data Seggregation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np

## Train Set

In [ ]:
X, y = np.arange(10).reshape((5,2)), range(5)

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

## KFold

In [ ]:
dataset= range(16)

In [ ]:
KFoldCrossValidator = KFold(n_splits=4, shuffle=False)
KFdataset = KFoldCrossValidator.split(dataset) # model will be trained in 4 patches

In [ ]:
print('{} {:61} {}'. format('Round', 'Training Set', 'Testing Set'))
for iteration, data in enumerate(KFdataset, start=1):
    print('{:^9} {} {:^25}'.format(iteration,data[0],str(data[1])))

# Training the Model

In [ ]:
import warnings
import pandas
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score 

In [ ]:
array = df.values
X = array[:,0:12] # first 12 columns to X
Y = array[:,12]

In [ ]:
max_error_scoring = 'max_error'
neg_mean_absolute_error_scoring = 'neg_mean_absolute_error'
r2_scoring = 'r2'
neg_mean_squared_error_scoring = 'neg_mean_squared_error'

In [ ]:
models = []
models.append(('LR', LinearRegression()))
models.append(('LASSO', Lasso()))
models.append(('EN',ElasticNet()))
models.append(('Ridge',Ridge()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
models.append(('SVR',SVR()))

In [ ]:
# Evaluate models and print results
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=7)
    cv_results = cross_val_score(model, X, Y, cv=kfold, scoring=max_error_scoring)
    cv_results2 = cross_val_score(model, X, Y, cv=kfold, scoring=neg_mean_absolute_error_scoring)
    cv_results3 = cross_val_score(model, X, Y, cv=kfold, scoring=r2_scoring)
    cv_results4 = cross_val_score(model, X, Y, cv=kfold, scoring=neg_mean_squared_error_scoring)
    msg = "%s: max error: %f , mean absolute error: %f, r2: %f, mean squared error: %f" % (name, cv_results.mean(), -cv_results2.mean(),cv_results3.mean(),-cv_results4.mean())
    print(msg)

In [ ]:
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1,shuffle=True)

In [ ]:
lasso_model  =Lasso()
lasso_model.fit(X_train,Y_train)

In [ ]:
predictions = lasso_model.predict(X_test)
print(predictions)

## Deploying the Model

In [ ]:
import pickle
pickle.dump(lasso_model, open('model.pkl', 'wb'))
model = pickle.load(open('model.pkl','rb'))

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
import requests
import numpy as np
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
@app.route('/predict',methods=['POST'])
def home():
    X = int(request.args.get('X', ''))
    Y = int(request.args.get('Y', ''))
    month = int(request.args.get('month', ''))
    day = int(request.args.get('day', ''))
    FFMC = float(request.args.get('FFMC', ''))
    DMC = float(request.args.get('DMC', ''))
    DC = float(request.args.get('DC', ''))
    ISI = float(request.args.get('ISI', ''))
    temp = float(request.args.get('temp', ''))
    RH = float(request.args.get('RH', ''))
    wind = float(request.args.get('wind', ''))
    rain = float(request.args.get('rain', ''))
    prediction = lasso_model.predict([[X, Y, month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain]])
    print('*******************************')
    print(prediction)
    return 'Prediction is ' + str(prediction[0])
      
app.run()